In [1]:
import pandas as pd
import numpy as np
import pymc3 as pm
import scipy.stats as stats
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.graph_objs import *

#### Code 3.1

In [2]:
PrPV = 0.95
PrPM = 0.01
PrV = 0.001
PrP = (PrPV*PrV + PrPM *(1 - PrV))
PrVP = PrPV * PrV/PrP
PrVP

0.08683729433272395

In [3]:
p_grid = np.linspace(start =0, stop = 1, num=20)
prior = np.repeat(1,20)
# compute likelihood at each value in grid
likelihood = stats.binom.pmf(6,9,p_grid)
posterior = likelihood * prior

# standardize the posterior, so it sums to 1
posterior = posterior / sum(posterior)
samples = np.random.choice(p_grid,p=posterior, size = int(1e4))

In [4]:
trace1 = go.Scatter(
         y= samples,
        mode = 'circles')
layout = go.Layout(
    autosize=False,
    width=500,
    height=500,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor='#7f7f7f',
    plot_bgcolor='#c7c7c7'
)
data = [trace1]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

### Creating the density plot

In [5]:
hist_data = [samples]
group_labels = ['proportion of water']
fig = ff.create_distplot(hist_data, group_labels,show_hist=False,show_rug=False)


fig.layout.update({'height':400})
fig.layout.update({'width':400})
py.iplot(fig)

In [6]:
sum(posterior[p_grid<0.5])

0.17072394266778282

In [7]:
sum(samples<0.5)/1e4

0.17349999999999999

In [8]:
sum((samples > 0.5) & (samples < 0.75)) / 1e4

0.62450000000000006

In [9]:
np.percentile(samples, 80)

0.78947368421052633

In [11]:
np.percentile(samples, [10,90])

array([ 0.42105263,  0.78947368])

### Code 3.1

In [15]:
p_grid = np.linspace(start =0, stop = 1, num=1000)
prior = np.repeat(1,1000)
# compute likelihood at each value in grid
likelihood = stats.binom.pmf(3,3,p_grid)
posterior = likelihood * prior

# standardize the posterior, so it sums to 1
posterior = posterior / sum(posterior)
samples = np.random.choice(p_grid,p=posterior, size = int(1e4))

### Code 3.2

In [16]:
np.percentile(samples, [25,75])

array([ 0.70570571,  0.93193193])

### Code 3.3

In [17]:
pm.hpd(samples, alpha=0.5)

array([ 0.84284284,  1.        ])